# Model Optimisation

This notebook performs hyperparameter optimisation for the six classical ML models across the nine feature-sets:
- RandomizedSearchCV
- GridSearchCV
- BayesSearchCV

In [1]:
import warnings
warnings.filterwarnings("ignore")
from pathlib import Path
import numpy as np
import pandas as pd
import joblib
import time
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV, cross_val_score
from skopt import BayesSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from scipy.stats import randint, uniform

BASE_DIR = Path.cwd().parents[1]
FEATURES_BASE = BASE_DIR / "features" / "04 - Fourth Working"
PROC_BASE = BASE_DIR / "results" / "04 - Fourth Working" / "Optimised"
MODEL_BASE = BASE_DIR / "models" / "04 - Fourth Working" / "Optimised"
FIG_BASE = BASE_DIR / "figures" / "04 - Fourth Working" / "Optimised"

for p in [PROC_BASE, MODEL_BASE, FIG_BASE]:
    p.mkdir(parents=True, exist_ok=True)

METHODS = ["rfe","skb","fscs","etc","pc","mi","mir","mu","vt"]

RANDOM_STATE = 42
CV = StratifiedKFold(n_splits=3, shuffle=True, random_state=RANDOM_STATE)

print("Configuration OK. Methods:", METHODS)

Configuration OK. Methods: ['rfe', 'skb', 'fscs', 'etc', 'pc', 'mi', 'mir', 'mu', 'vt']


## Hyperparameter Spaces

In [2]:
PARAM_DIST = {
    "Logistic Regression": {
        "C": uniform(0.001, 10),
        "penalty": ["l2"],
        "solver": ["lbfgs", "saga"],
        "max_iter": [500]
    },
    "Gradient Boosting": {
        "n_estimators": randint(50, 300),
        "learning_rate": uniform(0.01, 0.5),
        "max_depth": randint(2, 8),
        "subsample": uniform(0.5, 0.5)
    },
    "KNN": {
        "n_neighbors": randint(1, 31),
        "weights": ["uniform", "distance"],
        "p": [1, 2]
    },
    "Random Forest": {
        "n_estimators": randint(50, 400),
        "max_depth": randint(3, 20),
        "max_features": ["sqrt", "log2", None]
    },
    "Decision Tree": {
        "max_depth": randint(1, 20),
        "min_samples_split": randint(2, 20),
        "min_samples_leaf": randint(1, 20)
    },
    "SVM": {
        "C": uniform(0.01, 100),
        "kernel": ["rbf", "poly"],
        "gamma": ["scale", "auto"]
    }
}

PARAM_GRID = {
    "Logistic Regression": {
        "C": [0.01, 0.1, 1, 10],
        "solver": ["lbfgs"],
        "penalty": ["l2"]
    },
    "Gradient Boosting": {
        "n_estimators": [100, 200],
        "learning_rate": [0.01, 0.1],
        "max_depth": [3, 5]
    },
    "KNN": {
        "n_neighbors": [3,5,7,9],
        "weights": ["uniform","distance"],
        "p": [1,2]
    },
    "Random Forest": {
        "n_estimators": [100, 200],
        "max_depth": [None, 10, 20]
    },
    "Decision Tree": {
        "max_depth": [None, 5, 10],
        "min_samples_split": [2, 5, 10]
    },
    "SVM": {
        "C": [0.1, 1, 10],
        "kernel": ["rbf"],
        "gamma": ["scale"]
    }
}

BAYES_SPACE = {
    "Logistic Regression": {
        "C": (1e-3, 100.0, "log-uniform"),
        "solver": ["lbfgs"],
        "penalty": ["l2"]
    },
    "Gradient Boosting": {
        "n_estimators": (50, 300),
        "learning_rate": (0.01, 0.5, "log-uniform"),
        "max_depth": (2, 8)
    },
    "KNN": {
        "n_neighbors": (1, 31),
        "weights": ["uniform", "distance"],
        "p": [1, 2]
    },
    "Random Forest": {
        "n_estimators": (50, 400),
        "max_depth": (3, 30)
    },
    "Decision Tree": {
        "max_depth": (1, 30),
        "min_samples_split": (2, 50)
    },
    "SVM": {
        "C": (1e-2, 100.0, "log-uniform"),
        "kernel": ["rbf"],
        "gamma": ["scale", "auto"]
    }
}

## Helper Functions

In [3]:
def compute_metrics(y_true, y_pred):
    acc = accuracy_score(y_true, y_pred)
    prec = precision_score(y_true, y_pred, average="weighted", zero_division=0)
    rec = recall_score(y_true, y_pred, average="weighted", zero_division=0)
    f1 = f1_score(y_true, y_pred, average="weighted", zero_division=0)
    return {"Accuracy": acc, "Precision": prec, "Recall": rec, "F1": f1}

def save_confusion_matrix(y_true, y_pred, out_path, title):
    cm = confusion_matrix(y_true, y_pred)
    fig, ax = plt.subplots(figsize=(5,4))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=ax)
    ax.set_xlabel("Predicted"); ax.set_ylabel("True"); ax.set_title(title)
    fig.savefig(out_path, dpi=300, bbox_inches="tight")
    plt.close(fig)

def run_optimizer(name, model, param_space, X_train, y_train, optimizer_type="random", n_iter=30):
    if optimizer_type == "random":
        search = RandomizedSearchCV(model, param_distributions=param_space, n_iter=n_iter, cv=CV,
                                    scoring="accuracy", n_jobs=-1, random_state=RANDOM_STATE, verbose=0)
    elif optimizer_type == "grid":
        search = GridSearchCV(model, param_grid=param_space, cv=CV, scoring="accuracy", n_jobs=-1, verbose=0)
    elif optimizer_type == "bayes":
        search = BayesSearchCV(model, search_spaces=param_space, n_iter=n_iter, cv=CV, scoring="accuracy",
                               n_jobs=-1, random_state=RANDOM_STATE, verbose=0)
    else:
        raise ValueError("Unknown optimizer_type")
    start = time.time()
    search.fit(X_train, y_train)
    end = time.time()
    return search, end - start

## Model Definitions

In [4]:
MODELS = {
    "Logistic Regression": LogisticRegression(max_iter=1000, random_state=RANDOM_STATE),
    "Gradient Boosting": GradientBoostingClassifier(random_state=RANDOM_STATE),
    "KNN": KNeighborsClassifier(),
    "Random Forest": RandomForestClassifier(random_state=RANDOM_STATE),
    "Decision Tree": DecisionTreeClassifier(random_state=RANDOM_STATE),
    "SVM": SVC(probability=True, random_state=RANDOM_STATE)
}

## Main Optimisation Loop

In [5]:
n_iter_rand = 30
n_iter_bayes = 30

for method in tqdm(METHODS, desc="Methods"):
    feat_dir = FEATURES_BASE / method
    if not feat_dir.exists():
        print(f"⚠️ Missing feature folder for {method}; skipping.")
        continue

    train_df = pd.read_csv(feat_dir / "train.csv").dropna(subset=["DepressionEncoded"])
    test_df  = pd.read_csv(feat_dir / "test.csv").dropna(subset=["DepressionEncoded"])
    X_train = train_df.drop(columns=["DepressionEncoded"]).values
    y_train = train_df["DepressionEncoded"].astype(int).values
    X_test  = test_df.drop(columns=["DepressionEncoded"]).values
    y_test  = test_df["DepressionEncoded"].astype(int).values

    proc_out_base = PROC_BASE / method
    model_out_base = MODEL_BASE / method
    fig_out_base = FIG_BASE / method
    for p in [proc_out_base, model_out_base, fig_out_base]:
        p.mkdir(parents=True, exist_ok=True)

    rand_results = []
    grid_results = []
    bayes_results = []

    for model_name, model in tqdm(MODELS.items(), desc=f"Models ({method})", leave=False):
        print(f"\n=== {method.upper()} :: {model_name} ===")

        pdist = PARAM_DIST.get(model_name, {})
        pgrid = PARAM_GRID.get(model_name, {})
        bspace = BAYES_SPACE.get(model_name, {})

        try:
            print("-> RandomizedSearchCV (n_iter=", n_iter_rand, ")")
            search_rand, t_rand = run_optimizer(model_name, model, pdist, X_train, y_train, optimizer_type="random", n_iter=n_iter_rand)
            best_rand = search_rand.best_estimator_
            y_pred = best_rand.predict(X_test)
            metrics_rand = compute_metrics(y_test, y_pred)
            metrics_rand.update({"Method": method, "Model": model_name, "Optimizer": "Randomized", "TimeSec": t_rand})
            rand_results.append(metrics_rand)

            joblib.dump(best_rand, model_out_base / f"{model_name.lower().replace(' ','_')}_randomized.pkl")
            save_confusion_matrix(y_test, y_pred, fig_out_base / f"{model_name.lower().replace(' ','_')}_randomized_confusion.png",
                                  f"{model_name} Randomized ({method})")
            print(" Randomized done. Metrics:", metrics_rand)
        except Exception as e:
            print(" RandomizedSearchCV failed for", model_name, ":", e)

        try:
            print("-> GridSearchCV")
            if pgrid:
                search_grid, t_grid = run_optimizer(model_name, model, pgrid, X_train, y_train, optimizer_type="grid")
                best_grid = search_grid.best_estimator_
                y_pred = best_grid.predict(X_test)
                metrics_grid = compute_metrics(y_test, y_pred)
                metrics_grid.update({"Method": method, "Model": model_name, "Optimizer": "Grid", "TimeSec": t_grid})
                grid_results.append(metrics_grid)

                joblib.dump(best_grid, model_out_base / f"{model_name.lower().replace(' ','_')}_grid.pkl")
                save_confusion_matrix(y_test, y_pred, fig_out_base / f"{model_name.lower().replace(' ','_')}_grid_confusion.png",
                                      f"{model_name} Grid ({method})")
                print(" Grid done. Metrics:", metrics_grid)
            else:
                print("  No grid defined for", model_name, " — skipping GridSearch")
        except Exception as e:
            print(" GridSearchCV failed for", model_name, ":", e)

        try:
            print("-> BayesSearchCV (n_iter=", n_iter_bayes, ")")
            search_bayes, t_bayes = run_optimizer(model_name, model, bspace, X_train, y_train, optimizer_type="bayes", n_iter=n_iter_bayes)
            best_bayes = search_bayes.best_estimator_
            y_pred = best_bayes.predict(X_test)
            metrics_bayes = compute_metrics(y_test, y_pred)
            metrics_bayes.update({"Method": method, "Model": model_name, "Optimizer": "Bayes", "TimeSec": t_bayes})
            bayes_results.append(metrics_bayes)

            joblib.dump(best_bayes, model_out_base / f"{model_name.lower().replace(' ','_')}_bayes.pkl")
            save_confusion_matrix(y_test, y_pred, fig_out_base / f"{model_name.lower().replace(' ','_')}_bayes_confusion.png",
                                  f"{model_name} Bayes ({method})")
            print(" Bayes done. Metrics:", metrics_bayes)
        except Exception as e:
            print(" BayesSearchCV failed for", model_name, ":", e)

    if rand_results:
        pd.DataFrame(rand_results).to_csv(proc_out_base / "randomized_search_results.csv", index=False)
    if grid_results:
        pd.DataFrame(grid_results).to_csv(proc_out_base / "grid_search_results.csv", index=False)
    if bayes_results:
        pd.DataFrame(bayes_results).to_csv(proc_out_base / "bayes_search_results.csv", index=False)

    print(f"\nSaved results for method: {method} -> {proc_out_base}")

Methods:   0%|          | 0/9 [00:00<?, ?it/s]

Models (rfe):   0%|          | 0/6 [00:00<?, ?it/s]


=== RFE :: Logistic Regression ===
-> RandomizedSearchCV (n_iter= 30 )
 Randomized done. Metrics: {'Accuracy': 0.8296296296296296, 'Precision': 0.8300297344867618, 'Recall': 0.8296296296296296, 'F1': 0.8295904554605502, 'Method': 'rfe', 'Model': 'Logistic Regression', 'Optimizer': 'Randomized', 'TimeSec': 6.109628438949585}
-> GridSearchCV
 Grid done. Metrics: {'Accuracy': 0.8296296296296296, 'Precision': 0.8300297344867618, 'Recall': 0.8296296296296296, 'F1': 0.8295904554605502, 'Method': 'rfe', 'Model': 'Logistic Regression', 'Optimizer': 'Grid', 'TimeSec': 0.08954429626464844}
-> BayesSearchCV (n_iter= 30 )
 Bayes done. Metrics: {'Accuracy': 0.8296296296296296, 'Precision': 0.8300297344867618, 'Recall': 0.8296296296296296, 'F1': 0.8295904554605502, 'Method': 'rfe', 'Model': 'Logistic Regression', 'Optimizer': 'Bayes', 'TimeSec': 11.60192608833313}

=== RFE :: Gradient Boosting ===
-> RandomizedSearchCV (n_iter= 30 )
 Randomized done. Metrics: {'Accuracy': 0.7876543209876543, 'Preci

Models (skb):   0%|          | 0/6 [00:00<?, ?it/s]


=== SKB :: Logistic Regression ===
-> RandomizedSearchCV (n_iter= 30 )
 Randomized done. Metrics: {'Accuracy': 0.817283950617284, 'Precision': 0.8190286310747521, 'Recall': 0.817283950617284, 'F1': 0.8177827403320362, 'Method': 'skb', 'Model': 'Logistic Regression', 'Optimizer': 'Randomized', 'TimeSec': 1.0499181747436523}
-> GridSearchCV
 Grid done. Metrics: {'Accuracy': 0.817283950617284, 'Precision': 0.8186726327148609, 'Recall': 0.817283950617284, 'F1': 0.8174306846476589, 'Method': 'skb', 'Model': 'Logistic Regression', 'Optimizer': 'Grid', 'TimeSec': 0.09150457382202148}
-> BayesSearchCV (n_iter= 30 )
 Bayes done. Metrics: {'Accuracy': 0.817283950617284, 'Precision': 0.819014139351832, 'Recall': 0.817283950617284, 'F1': 0.8179392141796757, 'Method': 'skb', 'Model': 'Logistic Regression', 'Optimizer': 'Bayes', 'TimeSec': 15.55524730682373}

=== SKB :: Gradient Boosting ===
-> RandomizedSearchCV (n_iter= 30 )
 Randomized done. Metrics: {'Accuracy': 0.745679012345679, 'Precision': 

...


=== MIR :: Logistic Regression ===
-> RandomizedSearchCV (n_iter= 30 )
 Randomized done. Metrics: {'Accuracy': 0.8395061728395061, 'Precision': 0.8427692843089194, 'Recall': 0.8395061728395061, 'F1': 0.8396011896468398, 'Method': 'mir', 'Model': 'Logistic Regression', 'Optimizer': 'Randomized', 'TimeSec': 1.0732157230377197}
-> GridSearchCV
 Grid done. Metrics: {'Accuracy': 0.8444444444444444, 'Precision': 0.8473280919905855, 'Recall': 0.8444444444444444, 'F1': 0.8446967213181774, 'Method': 'mir', 'Model': 'Logistic Regression', 'Optimizer': 'Grid', 'TimeSec': 0.08527469635009766}
-> BayesSearchCV (n_iter= 30 )
 Bayes done. Metrics: {'Accuracy': 0.8444444444444444, 'Precision': 0.8473280919905855, 'Recall': 0.8444444444444444, 'F1': 0.8446967213181774, 'Method': 'mir', 'Model': 'Logistic Regression', 'Optimizer': 'Bayes', 'TimeSec': 12.340874910354614}

=== MIR :: Gradient Boosting ===
-> RandomizedSearchCV (n_iter= 30 )
 Randomized done. Metrics: {'Accuracy': 0.7975308641975308, 'Pre

Models (mu):   0%|          | 0/6 [00:00<?, ?it/s]


=== MU :: Logistic Regression ===
-> RandomizedSearchCV (n_iter= 30 )
 Randomized done. Metrics: {'Accuracy': 0.7925925925925926, 'Precision': 0.7946889804976945, 'Recall': 0.7925925925925926, 'F1': 0.792786493837252, 'Method': 'mu', 'Model': 'Logistic Regression', 'Optimizer': 'Randomized', 'TimeSec': 1.1547038555145264}
-> GridSearchCV
 Grid done. Metrics: {'Accuracy': 0.7925925925925926, 'Precision': 0.7946889804976945, 'Recall': 0.7925925925925926, 'F1': 0.792786493837252, 'Method': 'mu', 'Model': 'Logistic Regression', 'Optimizer': 'Grid', 'TimeSec': 0.0870518684387207}
-> BayesSearchCV (n_iter= 30 )
 Bayes done. Metrics: {'Accuracy': 0.7925925925925926, 'Precision': 0.7946889804976945, 'Recall': 0.7925925925925926, 'F1': 0.792786493837252, 'Method': 'mu', 'Model': 'Logistic Regression', 'Optimizer': 'Bayes', 'TimeSec': 15.6791250705719}

=== MU :: Gradient Boosting ===
-> RandomizedSearchCV (n_iter= 30 )
 Randomized done. Metrics: {'Accuracy': 0.8098765432098766, 'Precision': 0.

Models (vt):   0%|          | 0/6 [00:00<?, ?it/s]


=== VT :: Logistic Regression ===
-> RandomizedSearchCV (n_iter= 30 )
 Randomized done. Metrics: {'Accuracy': 0.8444444444444444, 'Precision': 0.846401606738586, 'Recall': 0.8444444444444444, 'F1': 0.8448222544644943, 'Method': 'vt', 'Model': 'Logistic Regression', 'Optimizer': 'Randomized', 'TimeSec': 0.862295389175415}
-> GridSearchCV
 Grid done. Metrics: {'Accuracy': 0.8444444444444444, 'Precision': 0.846401606738586, 'Recall': 0.8444444444444444, 'F1': 0.8448222544644943, 'Method': 'vt', 'Model': 'Logistic Regression', 'Optimizer': 'Grid', 'TimeSec': 0.07359886169433594}
-> BayesSearchCV (n_iter= 30 )
 Bayes done. Metrics: {'Accuracy': 0.8444444444444444, 'Precision': 0.846401606738586, 'Recall': 0.8444444444444444, 'F1': 0.8448222544644943, 'Method': 'vt', 'Model': 'Logistic Regression', 'Optimizer': 'Bayes', 'TimeSec': 11.473546981811523}

=== VT :: Gradient Boosting ===
-> RandomizedSearchCV (n_iter= 30 )
 Randomized done. Metrics: {'Accuracy': 0.8222222222222222, 'Precision': 

## Combine all optimizer results into a single report

In [6]:
combined_all = []
for method in METHODS:
    proc_out_base = PROC_BASE / method
    for fname in ["randomized_search_results.csv", "grid_search_results.csv", "bayes_search_results.csv"]:
        f = proc_out_base / fname
        if f.exists():
            df = pd.read_csv(f)
            df["SourceFile"] = fname
            combined_all.append(df)

if combined_all:
    combined_df = pd.concat(combined_all, ignore_index=True)
    combined_df.to_csv(BASE_DIR / "summary" / "results" / "Results Summary (Optimised)" / "fourth_working_results_summary.csv", index=False)
    display(combined_df.sort_values(["Model", "Optimizer"], ascending=True))
    print("Combined optimisation summary saved to:", BASE_DIR / "summary" / "results" / "Results Summary (Optimised)" / "fourth_working_results_summary.csv")
else:
    print("No optimisation results found to combine.")

,Accuracy,Precision,Recall,F1,Method,Model,Optimizer,TimeSec,SourceFile
16,0.740741,0.740555,0.740741,0.739826,rfe,Decision Tree,Bayes,11.999990,bayes_search_results.csv
34,0.696296,0.695105,0.696296,0.694870,skb,Decision Tree,Bayes,13.279467,bayes_search_results.csv
52,0.762963,0.764332,0.762963,0.761989,fscs,Decision Tree,Bayes,10.428710,bayes_search_results.csv
70,0.688889,0.685355,0.688889,0.686801,etc,Decision Tree,Bayes,14.747198,bayes_search_results.csv
88,0.691358,0.690779,0.691358,0.690138,pc,Decision Tree,Bayes,15.814460,bayes_search_results.csv
...,...,...,...,...,...,...,...,...,...
77,0.792593,0.791816,0.792593,0.790912,pc,SVM,Randomized,3.462590,randomized_search_results.csv
95,0.792593,0.791816,0.792593,0.790912,mi,SVM,Randomized,3.414855,randomized_search_results.csv
113,0.795062,0.796593,0.795062,0.795521,mir,SVM,Randomized,3.258403,randomized_search_results.csv
131,0.790123,0.793191,0.790123,0.790888,mu,SVM,Randomized,3.685384,randomized_search_results.csv


Combined optimisation summary saved to: d:\Programming\Projects\Depression Severity Assessment\summary\results\Results Summary (Optimised)\fourth_working_results_summary.csv
